In [0]:

import logging

# Specify the file path in your ADLS
new_chunks_path = "/mnt/silver-layer/new_chunks.parquet"

# Check if the file exists
try:
    files = dbutils.fs.ls(new_chunks_path)
    logging.info("File exists.")
except Exception as e:
    logging.error("No New Data Found")
    raise e

In [0]:
pip install azure-search-documents==11.4.0


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
new_chunks_df  = spark.read.parquet(new_chunks_path)


In [0]:
chunks = new_chunks_df.select("chunk_text").rdd.flatMap(lambda x: x).collect()

In [0]:
%pip install sentence-transformers


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from sentence_transformers import SentenceTransformer

# Load a local embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(chunks, batch_size=32, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2286 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [0]:
from pyspark.sql.types import *

embeddings_data_list =[]
for row , embbed_vec  in zip(new_chunks_df.rdd.collect(),embeddings):
    if not row.pub_date:
        continue
    
    data = {
        "embedd_vec_id":row.chunk_id,
        "pub_date": row.pub_date,
        "chunk_text": row.chunk_text,
        "start_index": row.start_index,
        "end_index":row.end_index,
        "embedding_vec": [float(x) for x in list(embbed_vec)]
    }
    embeddings_data_list.append(data)




In [0]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


# Initialize the SearchClient
service_endpoint = "https://iat-dp-vec-ser.search.windows.net"
admin_key = "gyliiZboxtLW4MFOalWeth2LASxf5HEpuWJVNjd0FGAzSeBzigEu"
index_name = "news_chunks_embedding_extra"

credential = AzureKeyCredential(admin_key)
search_client = SearchClient(service_endpoint, index_name, credential)



# Upload the documents
result = search_client.upload_documents(embeddings_data_list)

# Check if the upload was successful for each document
for res in result:
    if res.succeeded:
        print(f"Document with ID {res.key} uploaded successfully.")
        break
    else:
        print(f"Failed to upload document with ID {res.key}. Error message: {res.error_message}")


In [0]:


# # Specify the file path in your ADLS
chunks_path = "/mnt/silver-layer/processed_data/chunks.parquet"

# Check if the file exists
try:
    files = dbutils.fs.ls(chunks_path)

    #if it passed this step so there is a file 

    new_chunks_df.write.mode("append").parquet(chunks_path)
except Exception as e:
    new_chunks_df.write.parquet(chunks_path)


# remove the new_data file because it have been processed 
dbutils.fs.rm(new_chunks_path, recurse=True)

True

In [0]:
len(embeddings_data_list)

69000

In [0]:
# Define the schema
# schema = StructType([
#     StructField("embedd_vec_id", StringType(), True),
#     StructField("pub_date", DateType(), True),
#     StructField("chunk_text", StringType(), True),
#     StructField("start_index", IntegerType(), True),
#     StructField("end_index", IntegerType(), True),
#     StructField("embedding_vec", ArrayType(FloatType()), True),  # Embedding as array of floats
# ])

# # Create the Spark DataFrame
# embedding_df = spark.createDataFrame(embeddings_data_list, schema)

# # Show the DataFrame
# embedding_df.show(truncate=True)